In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#plt.rc("figure", dpi=300, figsize=(9,3))
plt.rc("figure", dpi=300)

<h1 align="center">Titanic Survival Estimation via Naïve Bayes</h1>

    Angela Cao
    Shane McQuarrie
    DRP Fall 2020

## Contents

1. Introduction: The Titanic Problem
2. Brief Data Summary
3. The Naïve Bayes Algorithm
4. Testing the Algorithm

## Introduction: The Titanic Problem

On April 15, 1912, the [RMS Titanic](https://en.wikipedia.org/wiki/Titanic) sank, leaving about 1,500 people dead. The goal of this project is to construct a machine learning algorithm to predict whether or not a person with given characteristics would have been likely to survive the accident. This is a [popular problem](https://www.kaggle.com/c/titanic) for first-time exposure to machine learning.

We have a [data set](https://www.kaggle.com/c/titanic/data) with information about 1,308 of the Titanic passengers, with the following features for each passenger.
- `Pclass`: The class of the ticket, either 1 for 1st class (most expensive), 2 for 2nd class, and 3 for 3rd class (least expensive).
- `Survived`:
- `Name`:
- `Sex`: 
- `Age`:
- `Sibsp`:
...

**Angela, see https://www.kaggle.com/c/titanic/data for column descriptions if you want to include them.**

In [ ]:
# Read the data from file.
titanic_original = pd.read_csv("titanic.csv")
titanic_original.sample(5)

Displays 5 random entries from the original Titanic data. 

**(what columns do we care about?)**

What are the common factors among survivors and victims of the Titanic accident? We will construct a classifier based on the following features: `Sex`, `Age`, `Pclass`, `Fare`. (Your `Name` is independent of whether or not you survive, but your age may be a relevant factor!)

In [ ]:
# Extract the relevant columns.
titanic = titanic_original[["Survived", "Sex", "Age", "Pclass", "Fare"]]
titanic.sample(5)

Displays 5 random entries from the recently-reformed Titanic data. 

We split the data into training and testing sets, reserving 15% of the entries for testing.

**(how are we dealing with missing data?)**

The data is incomplete in that there are missing entries for some of the features for some of the passengers. To remedy this, we fill ...

In [ ]:
# Split the data into training and testing sets.
train_data, test_data = train_test_split(titanic, test_size=.15)
print(f"{len(train_data)} train points, {len(test_data)} test points")

# Minor preprocessing step: replace NaN values with averages.
filler = train_data.mean()
train_data = train_data.fillna(filler)

# Separate the Survival labels from the test data and replace
# missing values with the averages from the training data.
test_labels = test_data["Survived"]
test_data = test_data.fillna(filler).drop("Survived", axis=1)

test_data.sample(3)

Displays 3 random entries from the Titanic test data. 

## Brief Data Summary

**Principles to follow here**:
- Tell a story. That means only use tables/visualizations that serve the story.
    - How many people survived?
    - How does survival appear to depend on the other factors (subgroups?)
- Visualize single distributions with histograms / kernel density estimate (`kind="kde"`)
- Label everything!


Ideas for visualizations
- Proportion of people who died
- Distribution of ages?
- Distribution of Fare?
- Proportion of males / females
- Any of these grouped by survival

(go see what we did in the Pandas 3 lab)

Many of the victims (and survivors) shared certain characteristics such as class (divided into 3 classes based on social status), sex, age, etc. We begin by investigating the common factors among survivors and victims of the Titanic sinking. 

In [ ]:
titanic.groupby(["Survived", "Sex"]).mean()

The data is first sorted into who survived the Titanic incident and who didn't (0 for non-survivors and 1 for survivors), and then sorted within these survival groups based on sex (female/male) and got the mean data of the female/male survivors/non-survivors.

From the table, we can see that:
- The mean age of female non-survivors is around 25 years old. A good amount of the dead females, on average (mean), bought 2nd and 3rd class tickets on average (mean) of 22.32 dollars. 
- The mean age of male non-survivors is around 31-32 years old. A good amount of the dead males, on average (mean), bought 2nd and 3rd class tickets on average (mean) of 23.55 dollars. 
- The mean age of female survivors is around 29-30 years old. A good amount of the dead females, on average (mean), bought 1st and 2nd class tickets on average (mean) of 55.14 dollars. 
- The mean age of male survivors is around 26-27 years old. A good amount of the dead males, on average (mean), bought second class tickets on average (mean) of 37.19 dollars. 

In [ ]:
titanic.groupby("Survived").count()["Age"].plot(kind="barh")
plt.show()

The bar graph above shows how many passengers from the Titanic data survived (1) and how many passengers from the Titanic data did not (0). It seems more passengers recorded in the data perished than survived. 

In [ ]:
titanic.groupby("Pclass").count()["Age"].plot(kind="barh")
plt.show()

The bar graph above shows the number of passengers in each ticket class recorded in the data. The majority of the passengers recorded in the data bought 3rd class tickets. 

In [ ]:
titanic.groupby("Sex").count()["Age"].plot(kind="barh")
plt.show()

The bar graph above shows the number of female and male passengers respectiviely recorded in the data. The data has more male than female passengers recorded. 

In [ ]:
titanic["Age"].plot(kind="hist", bins=80)
plt.show()

The histogram above shows the distribution of the ages of the passengers in the Titanic data in 80 bins or groups. From the histogram, you can see that the data is right-skewed with a good amount of passengers in their 20's and 30's and also a good chunk of children as well. The amount of passengers each group increases from 10's to early-mid 20's and then increases afterwards with some outliers in 70's and 80's groups. 

In [ ]:
titanic["Fare"].plot(kind="hist", bins=20)
plt.show()

The histogram above shows the distribution of the fare prices of the passengers in the Titanic data in 20 bins or groups. From the histogram you can see the data is extremely right-skewed with the majority of passengers having tickets that cost 100 dollars or less. The amount of passengers in each group decreases every time until we have at least a single outlier for 500 dollar ticket. 

In [ ]:
titanic.describe()

Not only shows the amount of passengers recorded in the data through the 'Survived' column (1309 passengers recorded, and most of them probably perished according to the mean and quartiles), but also the number of passengers in the data who got their ages recorded and the mean and ranges of ages recorded, and the mean and ranges of classes and fares recorded as well through min, quartiles, and max. 

From the table, we can see that: 
- The majority of passengers recorded perished in the Titanic incident. 
- There were 1046 passengers whose ages were recorded in the data. Some passengers are missing ages. 
- The mean age of the passsengers in the Titanic were 29-30 years old. 
- The median age of the passengers in the Titanic is about 28 years old which confirms the right-skewness of the distribution of ages. 
- The youngest passenger was a baby while the oldest passenger is 80 years old. 
- A good majority of passengers recorded bought 3rd class tickets though the mean (average) was is about 2nd class. 
- The mean fare of the passengers in the Titanic was about 33.30 dollars. 
- The median fare of the passengers in the Titanic was about 14.45 dollars which confirms the extreme right-skewness of the distribution of fare. 
- The cheapest ticket was for free while the most expensive ticket was about 512.33 dollars. 

In [ ]:
titanic["Age"].isnull()

In [ ]:
titanic.loc[titanic["Age"].isnull()]

The table above shows all the passengers in the data whose ages are unknown so far. According to the table, there are 263 entries or passengers whose ages have not been recorded yet. We decide to get the median age of all passengers as the "age" of these passengers as of now to keep the data as consistent as we can. 

In [ ]:
titanic["Age"].median()

In [ ]:
titanic.loc[titanic["Age"].isnull(), "Age"] = titanic["Age"].median()

In [ ]:
titanic = titanic[:1309]

In [ ]:
plt.hist(titanic["Age"])

The histogram above shows the distribution of the ages of the passengers in the Titanic data in 10 bins or groups. From the histogram, you can see that the data is right-skewed with a good amount of passengers in their 20's and 30's and also a good chunk of children as well. 

In [ ]:
plt.hist(titanic["Fare"])

The histogram above shows the distribution of the fare prices of the passengers in the Titanic data in 10 bins or groups. From the histogram you can see the data is extremely right-skewed with the majority of passengers having tickets that cost approximately 150 dollars or less. The amount of passengers in each group decreases every time until we have many outliers in the 200's, 300's, and 500's groups.  

**Survived Classification Data Summary**

In [ ]:
survival_groups = titanic.groupby("Survived")

In [ ]:
survival_groups.boxplot(grid=False, column=['Age'], vert=False)

In [ ]:
survival_groups.plot(kind='hist', y='Age')

In [ ]:
survival_groups.boxplot(grid=False, column=['Fare'], vert=False)

In [ ]:
survival_groups.plot(kind="hist", y="Fare")

**Pclass Classification Data Summary**

In [ ]:
groups = titanic.groupby("Pclass")

In [ ]:
# These almost look nice, but we have to be careful not compare apples and oranges.
groups.boxplot(grid=False, column=['Age'], vert=False)

In [ ]:
groups.plot(kind="hist", y="Age")

In [ ]:
groups.boxplot(grid=False, column=['Fare'], vert=False)

In [ ]:
groups.plot(kind="hist", y="Fare")

**Sex Classification Data Summary**

In [ ]:
gender_groups = titanic.groupby("Sex")

In [ ]:
gender_groups.boxplot(grid=False, column=['Age'], vert=False)

In [ ]:
gender_groups.plot(kind="hist", y="Age")

In [ ]:
gender_groups.boxplot(grid=False, column=['Fare'], vert=False)

In [ ]:
gender_groups.plot(kind="hist", y="Fare")

**Age and Fare as Classification**

In [ ]:
# Partition the passengers into 3 categories based on age.
age = pd.cut(titanic['Age'], [0, 12, 18, 80])
age

In [ ]:
titanic.pivot_table(values='Survived', index=['Sex', age],
                   columns='Pclass', aggfunc='mean')

In [ ]:
titanic.pivot_table(values='Survived', index=['Sex', age], 
                   columns='Pclass', aggfunc='count')

In [ ]:
# Cut the ticket price into two intervals (cheap vs expensive)
fare = pd.qcut(titanic['Fare'], 2)
fare

In [ ]:
titanic.pivot_table(values='Survived',
                   index=['Sex', age], columns=[fare, 'Pclass'], 
                   aggfunc='count', fill_value='-')

**Age and Fare Correlation**

In [ ]:
titanic.plot(kind='box', y=['Age'], vert=False)
titanic.plot(kind='box', y=['Fare'], vert=False)

In [ ]:
titanic.plot(kind='scatter', x='Age', y='Fare', alpha=.2)

In [ ]:
first_class = titanic[titanic['Pclass'] == 1.0]
first_class.plot(kind='box', y=['Age'], vert=False)
first_class.plot(kind='box', y=['Fare'], vert=False)
first_class.plot(kind='scatter', x='Age', y='Fare')

## The Naïve Bayes Algorithm

Our goal is to construct a function $f$ that maps a given tuple of `Sex`, `Age`, `Pclass`, and `Fare` values to a `Survival` value (0 for perished or 1 for survived).
This is a _binary classification problem_ because we are separating instances of data into two categories.
Since we have labeled data
There are many ways to construct such a function, but we will use the simple Naïve Bayes algorithm.

Let $P$ be a probability measure and let $A$ and $B$ be events in a probability space. _Bayes' rule_ is the statement

$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}.
$$

We want to compute the probability of survival given the features in the data, so we want to compute
$$
P(\textrm{Survived}|(\textrm{Sex,Age,Pclass,Fare})) = \frac{P((\textrm{Sex,Age,Pclass,Fare})|\textrm{Survived})P(\textrm{Survived})}{P((\textrm{Sex,Age,Pclass,Fare}))}.
$$

For notational convenience, let $X_1,\ldots,X_4$ denote the four features and $Y$ denote the survival label. Then our statement is
$$
P(Y|X_1,X_2,X_3,X_4) = \frac{P(X_1,X_2,X_3,X_4|Y)P(Y)}{P(X_1,X_2,X_3,X_4)}.
$$

The probability $P(Y)$ is called the _prior_ **(choose this to be .5 or something else, and explain why)**.

The denominator $P(X_1,X_2,X_3,X_4)$ is called the _evidence_, which we write with the Law of Total Probability:

$$
P(X_1,X_2,X_3,X_4) = \sum_{i\in\{0,1\}} P(X_1,X_2,X_3,X_4|Y=i)P(Y=i).
$$

So what we really want is to compute $P(X_1,X_2,X_3,X_4|Y=i)$. Here we use the (naïve) assumption that **the features are conditionally independent**, that is,

$$
\begin{align*}
    P(X_1,X_2,X_3,X_4|Y=i)
    &= \prod_{j=1}^{4}P(X_{j}|Y = i)
    \\
    &= P(X_1|Y=i)P(X_2|Y=i)P(X_3|Y=i)P(X_4|Y=i)
\end{align*}
$$

To compute each $P(X_j|Y=i)$, we need to choose a probabilty distribution to evaluate. For example, Sex has two values, Male and Female. So we choose a Bernoulli distribution to model these probabilities, i.e.,

$$
\begin{align*}
    P(X_2 = \textrm{Male} | Y = i) &= q_i
    &
    P(X_2 = \textrm{Female} | Y = i) &= 1 - q_i
\end{align*}
$$
And we compute $q_i$ from the data.

#### Implementation

In [ ]:
class TitanicNaiveBayes:
    """Naïve Bayes classifier for the Titanic problem, mapping values of
    Sex, Age, Fare, and Pclass to Survival.
    """
    def fit(self, data):
        """Calculate statistics for each group based on survival.
        
        Parameters
        ----------
        data : pd.DataFrame
            Data to train on.
        """
        for i in range(0, 2):
            total = 0
            males = 0
            mean = 0.0
            ages = []
            std = 0.0
            for k in range(0, len(data)):
                status = labels[k]
                if status == i:
                    total += 1
                    if data.iloc[k][0] == 'male':
                        males += 1
                    mean += data.iloc[k][1]
                    ages.append(data.iloc[k][1])
            probability = (total * 1.0) / len(data)
            males = (males * 1.0) / (total * 1.0)
            mean = (mean * 1.0) / (total * 1.0)
            for k in range(0, len(ages)):
                std += pow((ages[k] - mean), 2)
                std = std / (total * 1.0)
                std = sqrt(std)
            print("Group "+str(i))
            print("Probability = "+str(probability))
            print("Mean = "+str(mean))
            print("Standard Deviation = "+str(std))
        return self

    def predict(self, data):
        """Calculate the probabilities of belonging to each distribution.
    
        Parameters
        ----------
        data: pd.DataFrame
            Data to train on.
        """        
        denom = 0
        prob_group = [0.0, 0.0]
        for i in range(0, 2):
            females = 0
            males = 0
            mean = 0.0
            ages = []
            std = 0.0
            total = 0
            for k in range(0, len(data)):
                status = labels[k]
                if status == i:
                    total += 1
                    if data.iloc[k][0] == 'male':
                        males += 1
                    else:
                        females += 1
                    mean += data.iloc[k][1]
                    ages.append(data.iloc[k][1])
            females = (females * 1.0) / (total * 1.0)
            males = (males * 1.0) / (total * 1.0)
            mean = (mean * 1.0) / (total * 1.0)
            for k in range(0, len(ages)):
                std += pow((ages[k] - mean), 2)
                std = std / (total * 1.0)
                std = sqrt(std)
            ### idk about the rest lol
        pass

In [ ]:
class TitanicNaiveBayes:
    """Naïve Bayes classifier for the Titanic problem, mapping values of
    Sex, Age, Fare, and Pclass to Survival.
    """
    def fit(self, data):
        """Calculate statistics for each group based on survival.
        
        Parameters
        ----------
        data : pd.DataFrame
            Data to train on.
        """
        groups = data.groupby("Survived")
        
        sex_params = {}
        age_params = {}
        fare_params = {}
        pclass_params = {}
        
        for Y, group in groups:
            # Get the average number of males in the group.
            sex_params[Y] = np.mean(group["Sex"] == "male")

            # Get the mean and standard deviation of the ages in the group.
            age_params[Y] = np.mean(group["Age"]), np.std(group["Age"])
            
            # Get the mean and standard deviation of the fare in the group.
            fare_params[Y] = np.mean(group["Fare"]), np.std(group["Fare"])
            
            # Get the probability for each class in the group.
            pclass_params[Y] = [np.mean(group["Pclass"] == j) for j in [1,2,3]]
        
        self.sex_params_ = sex_params
        self.age_params_ = age_params
        self.fare_params_ = fare_params
        self.pclass_params_ = pclass_params
        return self

    def predict(self, data):
        """Calculate the probabilities of belonging to each distribution.
    
        Parameters
        ----------
        data: pd.DataFrame
            Data to train on.
        """
        probabilities = []
        for row in data.values:
            numerators = []
            for Y in [0,1]:
                q = self.sex_params_[Y]
                p1 = q if data["Sex"] == "male" else 1 - q

                m2, s2 = self.age_params_[Y]
                p2 = stats.norm(m2, s2).pdf(data["Age"])

                m2, s2 = self.age_params_[Y]
                p3 = self.age_params_[Y]

                qs = self.pclass_params_[Y]
                p4 = qs[int(data["Pclass"]) - 1]

                numerators.append(np.exp(np.sum(np.log([p1, p2, p3, p4, .5]))))

            denom = sum(numerators)
            probabilities.append(np.array(numerators) / denom)
        return probabilities

In [ ]:
class TitanicNaiveBayes2:
    """Naïve Bayes classifier for the Titanic problem, mapping values of
    Sex, Age, Fare, and Pclass to Survival (0 = perished, 1 = survived).
    
    Attributes
    ----------
    prior_ : scipy.stats Bernoulli distribution
        Survival probability prior, i.e., prior_.pmf(Y) returns the
        probability of perishing if Y = 0 or surviving if Y = 1.
    
    likelihood_ : dict[Y -> scipy.stats distributions]
        Independent likelihood probability distributions whose parameters
        are computed from the training data. In order:
        * Sex: Bernoulli
        * Age: Gaussian (normal)
        * Fare: Exponential
        * Pclass: Categorical
    """
    def __init__(self, survival_prior=.5):
        """Set the probability of survival (Benoulli prior distribution)."""
        self.prior_ = stats.bernoulli(survival_prior)

    def fit(self, data):
        """Calculate statistics for each group based on survival.
        
        Parameters
        ----------
        data : pd.DataFrame
            Data to train on. Must have columns
            ["Survived", "Sex", "Age", "Fare", "Pclass"].
        
        Returns
        -------
        self
        """
        groups = data.groupby("Survived")
        self.likelihood_ = {}
        
        for Y, group in groups:
            # Record the label.
            distributions = {}
            
            # Sex distribution: Bernoulli.
            q = np.mean(group["Sex"] == "male")
            distributions["Sex"] = stats.bernoulli(q)
            
            # Age distribution: Gaussian (Normal).
            µ, σ = np.mean(group["Age"]), np.std(group["Age"])
            distributions["Age"] = stats.norm(µ, σ)

            # Fare distribution: Exponential.
            loc, scale = stats.expon.fit(group["Fare"])
            distributions["Fare"] = stats.expon(loc, scale)
            
            # Pclass distribution: Categorical.
            qs = np.array([np.mean(group["Pclass"] == j) for j in [1,2,3]])
            distributions["Pclass"] = qs
            
            self.likelihood_[Y] = distributions
        
        return self

    def proba(self, data):
        """Calculate the probabilites of each row of data belonging to each
        survival category via Bayes' rule:
        
                                        P(features | Survived=i) P(Survived)
        P(Survived=i | features) = ---------------------------------------------
                                   sum_{j}[P(features | Survived=j) P(Survived)]
        
        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].
        
        Returns
        -------
        probabilities : (m,2) ndarray
            Perish / survival probabilities.
        """
        numerators = []
        for Y in self.likelihood_:
            dist = self.likelihood_[Y]

            # Evaluate the likelihood and prior distributions.
            pXs = [dist["Sex"].pmf(data["Sex"] == "male"),        # P(Sex|Y)
                   dist["Age"].pdf(data["Age"]),                  # P(Age|Y)
                   dist["Fare"].pdf(data["Fare"]),                # P(Fare|Y)
                   dist["Pclass"][np.int32(data["Pclass"]) - 1],  # P(Pclass|Y)
                   self.prior_.pmf([Y]*len(data))                 # P(Y)
                   ]

            numerators.append(np.exp(np.sum(np.log(pXs), axis=0)))
        numers = np.array(numerators)

        return np.transpose(numers / np.sum(numers, axis=0))
    
    def predict(self, data):
        """Predict a survival category for each row of data via Bayes' rule.
        
        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].
        
        Returns
        -------
        labels : (m,) ndarray
            Perish / survival probabilities.
        """
        return np.argmax(self.proba(data), axis=-1)

    def accuracy(self, data, labels):
        """Compute the accuracy of the model given some test data.
        
        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        accuracy : float
            The % of rows guessed correctly.
        """
        return np.mean(self.predict(data) == labels)

    # Distribution plots ------------------------------------------------
    def _survival_label(self, Y):
        return "Perished" if Y == 0 else "Survived"

    def plot_prior(self):
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["Prior"])
        data["Prior"] = self.prior_.pmf([0,1])
        data.plot(kind="barh")
        plt.legend([])
        plt.title("Prior Survival Distribution")

    def plot_sex(self):
        sexes = np.array([0, 1])
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["Female", "Male"])
        for Y, distributions in self.likelihood_.items():
            data.loc[self._survival_label(Y)] = distributions["Sex"].pmf(sexes)
        data.plot(kind="barh")
        plt.legend(loc="upper right")
        plt.title("Estimated Sex Distributions")

    def plot_age(self):
        years = np.linspace(0, 100, 1000)
        for Y, distributions in self.likelihood_.items():
            plt.plot(years, distributions["Age"].pdf(years),
                     label=self._survival_label(Y))
        plt.legend(loc="upper right")
        plt.title("Estimated Age Distributions")

    def plot_fare(self):
        fares = np.linspace(0, 550, 1000)
        for Y, distributions in self.likelihood_.items():
            plt.plot(fares, distributions["Fare"].pdf(fares),
                     label=self._survival_label(Y))
        plt.legend(loc="upper right")
        plt.title("Estimated Fare Distributions")

    def plot_pclass(self):
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["1st Class", "2nd Class", "3rd Class"])
        for Y, distributions in self.likelihood_.items():
            data.loc[self._survival_label(Y)] = distributions["Pclass"]
        data.plot(kind="barh")
        plt.legend(loc="upper right")
        plt.title("Estimated Pclass Distributions")
    
    def plot_likelihoods(self):
        """Plot each of the learned distributions."""
        self.plot_prior()
        plt.show()
        self.plot_sex()
        plt.show()
        self.plot_age()
        plt.show()
        self.plot_fare()
        plt.show()
        self.plot_pclass()
        plt.show()

## Applying the Algorithm to the Problem

Before we use Naïve Bayes on the Titanic data, we should check the assumption of conditional independence among the training variables.

In [ ]:
# Are these variables correlated?
train_data.drop("Survived", axis=1).corr()

In [ ]:
nb = TitanicNaiveBayes2(.5).fit(train_data)
nb.plot_likelihoods()

**Angela: can you interpret what's going on here? What story is it telling? :)**

In [ ]:
nb.accuracy(test_data, test_labels)

In [ ]:
samples = test_data.sample(5)
samples

In [ ]:
nb.proba(samples)

In [ ]:
nb.predict(samples)

In [ ]:
test_data.iloc[np.argmax(nb.proba(test_data), axis=0)]